Discovery of proteoforms with S/N more than 10, and matching them with their charge states

In [26]:
# -*- coding: utf-8 -*-
"""
Pipeline:
1) Detect neutral-mass peaks from a deconvoluted spectrum (mass intensity; whitespace- or csv-delimited).
2) Use detected peaks as candidate proteins and assign raw MS1 peaks by charge-series matching.

Outputs:
- <deconv_stem>_detected_signals.csv / .png   (neutral-mass peak picks + metadata, includes SNR)
- OUT_DIR/assigned_ms1_with_peaks.csv         (annotated raw MS1 with assigned_mass/charge)
- OUT_DIR/assignments_summary.csv             (one row per accepted neutral mass; now includes SNR)
- OUT_DIR/<several_plots>.png                 (mirror plots + neutral-mass spectrum)
"""

from __future__ import annotations
import os
import re
import json
from dataclasses import dataclass
from pathlib import Path
from bisect import bisect_left

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks


# ============================================================
# --------------------  PEAK DETECTION  ----------------------
# ============================================================

@dataclass
class PeakFindingParams:
    min_prominence: float | None = None
    min_height: float | None = None
    min_distance_pts: int = 10
    smooth_window: int = 0
    min_snr: float = 0.0


def _mad_sigma(y: np.ndarray) -> float:
    if y.size == 0:
        return 0.0
    med = np.median(y)
    mad = np.median(np.abs(y - med))
    return 1.4826 * mad


def _smooth(y: np.ndarray, window: int) -> np.ndarray:
    if window < 3 or window % 2 == 0:
        return y
    kernel = np.ones(window, dtype=float) / window
    return np.convolve(y, kernel, mode="same")


def _extract_id_list(name: str, key: str) -> list[int] | None:
    """
    Extract an underscore- or hyphen-separated list of integers after a key.
    Examples:
      "__pos_3__"          -> [3]
      "__neg_0_1_2_"       -> [0,1,2]
      "-pos-10-11"         -> [10,11]
    """
    m = re.search(rf"(?:^|[_-]){key}((?:[_-]\d+)+)(?=[_-]|$)", name, flags=re.I)
    if not m:
        m1 = re.search(rf"(?:^|[_-]){key}[_-]?(\d+)(?=[_-]|$)", name, flags=re.I)
        if m1:
            return [int(m1.group(1))]
        return None
    parts = re.findall(r"\d+", m.group(1))
    return [int(x) for x in parts] if parts else None


def parse_metadata_from_filename(path: str | Path) -> dict:
    """
    Extract bin, experiments/controls (IDs + counts), regulation, replicate, source_file.
    Regulation token is taken as the LAST occurrence among (negabs|posabs|neg|pos).
    """
    p = Path(path)
    name = p.stem

    meta = {
        "bin": None,
        "experiments": None,
        "controls": None,
        "experiments_ids": None,
        "controls_ids": None,
        "experiments_n": None,
        "controls_n": None,
        "regulation": None,
        "replicate": None,
        "source_file": p.name,
    }

    # bin: "bin5"/"bin_5" or a leading number "75__pos_..."
    m = re.search(r"(?:^|[_-])bin[_-]?(\d+)(?=[_-]|$)", name, flags=re.I)
    if m:
        meta["bin"] = int(m.group(1))
    else:
        m2 = re.match(r"^(\d+)(?=[_-])", name)
        if m2:
            meta["bin"] = int(m2.group(1))

    exp_ids = _extract_id_list(name, "pos")
    ctl_ids = _extract_id_list(name, "neg")
    if exp_ids is not None:
        meta["experiments_ids"] = ",".join(str(x) for x in exp_ids)
        meta["experiments_n"] = len(exp_ids)
        meta["experiments"] = len(exp_ids)
    if ctl_ids is not None:
        meta["controls_ids"] = ",".join(str(x) for x in ctl_ids)
        meta["controls_n"] = len(ctl_ids)
        meta["controls"] = len(ctl_ids)

    reg_tokens = [m.group(1).lower() for m in re.finditer(
        r"(?:^|[_-])(negabs|posabs|neg|pos)(?=[_-]|$)", name, flags=re.I
    )]
    if reg_tokens:
        token = reg_tokens[-1]
        reg_map = {"negabs": "downregulated", "neg": "downregulated",
                   "posabs": "upregulated", "pos": "upregulated"}
        meta["regulation"] = reg_map.get(token)

    m = re.search(r"(?:^|[_-])run([A-Za-z])(?=[_-]|$)", name)
    if m:
        meta["replicate"] = m.group(1).upper()

    return meta


def load_space_separated(path: str | Path) -> pd.DataFrame:
    path = Path(path)
    # whitespace-delimited by default; fallback to CSV
    try:
        df = pd.read_csv(path, sep=r"\s+", engine="python", header=None,
                         names=["mass", "intensity"], comment="#")
    except Exception:
        df = pd.read_csv(path, header=None)
        if df.shape[1] >= 2:
            df = df.iloc[:, :2]
            df.columns = ["mass", "intensity"]
        else:
            raise ValueError("Deconvoluted file must have at least two columns: mass intensity")
    return df


def detect_signals(
    df: pd.DataFrame,
    params: PeakFindingParams = PeakFindingParams()
) -> pd.DataFrame:
    # normalize columns
    if not {"mass", "intensity"}.issubset(df.columns):
        if df.shape[1] >= 2:
            df = df.copy()
            df.columns = ["mass", "intensity"] + [f"col{i}" for i in range(2, df.shape[1])]
        else:
            raise ValueError("Input DataFrame must have columns ['mass','intensity'].")

    df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["mass", "intensity"])
    df = df.sort_values("mass").reset_index(drop=True)

    x = df["mass"].to_numpy(float)
    y = df["intensity"].to_numpy(float)

    y_proc = _smooth(y, params.smooth_window)

    sigma = _mad_sigma(y_proc)
    ymax = float(np.max(y_proc)) if y_proc.size else 0.0

    min_prom = params.min_prominence or max(6.0 * sigma, 0.001 * ymax)
    min_h    = params.min_height     or max(4.0 * sigma, 0.0005 * ymax)

    peaks, props = find_peaks(
        y_proc,
        prominence=min_prom,
        height=min_h,
        distance=max(1, int(params.min_distance_pts))
    )

    out = pd.DataFrame({
        "mass": x[peaks],
        "intensity": y[peaks],
        "prominence": props.get("prominences", np.full(peaks.shape, np.nan)),
        "left_base_idx": props.get("left_bases", np.full(peaks.shape, -1)),
        "right_base_idx": props.get("right_bases", np.full(peaks.shape, -1)),
    })

    # SNR estimate and filter
    snr_den = sigma if sigma > 0 else (np.std(y_proc) if y_proc.size else 1.0)
    snr_den = snr_den if snr_den > 0 else 1.0
    out["snr"] = out["intensity"] / snr_den

    if params.min_snr > 0:
        out = out[out["snr"] >= params.min_snr].reset_index(drop=True)

    return out.sort_values("intensity", ascending=False).reset_index(drop=True)


def plot_spectrum_with_peaks(
    df: pd.DataFrame,
    peaks_df: pd.DataFrame,
    out_png: str | Path | None = None,
    title: str = "Detected Neutral-Mass Signals"
) -> None:
    plt.figure(figsize=(10, 4))
    plt.plot(df["mass"].to_numpy(), df["intensity"].to_numpy(), linewidth=1)
    if peaks_df is not None and not peaks_df.empty:
        plt.scatter(
            peaks_df["mass"].to_numpy(),
            peaks_df["intensity"].to_numpy(),  # use peaks’ intensities
            s=18
        )
    plt.xlabel("Neutral mass (Da)")
    plt.ylabel("Intensity (arb.)")
    plt.title(title)
    plt.tight_layout()
    if out_png:
        plt.savefig(out_png, dpi=150)
    plt.close()


# ============================================================
# -----------------  CHARGE-SERIES MATCHING  -----------------
# ============================================================

# Matching parameters (tweak as needed)
PROTON_MASS = 1.007276466812  # Da

Z_MIN, Z_MAX = 5, 50
PPM_TOL = 1000.0              # ppm window for m/z match
ABS_DA_TOL = 1.0              # absolute Da floor (used with ppm)
MIN_MATCHED_CHARGE_STATES = 4 # require ≥ N charge-state hits to accept a protein


def _read_raw_ms1(path: str) -> pd.DataFrame:
    """
    Robustly read raw MS1 CSV. Expected two columns (m/z, intensity), with or without headers.
    If more columns exist, pick the best 'mz' and 'intensity' columns.
    """
    try:
        df = pd.read_csv(path)
    except Exception:
        df = pd.read_csv(path, header=None)

    if df.shape[1] == 2:
        df.columns = ["mz", "intensity"]
    else:
        cols_lower = [str(c).lower() for c in df.columns]
        mz_candidates = [i for i, c in enumerate(cols_lower)
                         if ("mz" in c) or ("m/z" in c) or ("mass/charge" in c) or (c.strip() == "m z")]
        int_candidates = [i for i, c in enumerate(cols_lower)
                          if ("int" in c) or ("abund" in c) or ("height" in c) or ("signal" in c)]
        if not mz_candidates:
            mz_candidates = [0]
        if not int_candidates:
            int_candidates = [1 if df.shape[1] > 1 else 0]
        df = df.iloc[:, [mz_candidates[0], int_candidates[0]]].copy()
        df.columns = ["mz", "intensity"]

    df = df.replace([np.inf, -np.inf], np.nan).dropna()
    df = df[df["intensity"] > 0].copy()
    df["mz"] = pd.to_numeric(df["mz"], errors="coerce")
    df["intensity"] = pd.to_numeric(df["intensity"], errors="coerce")
    df = df.dropna().sort_values("mz").reset_index(drop=True)
    return df


def _ppm_window(target_mz: float, ppm: float, abs_da: float) -> tuple[float, float]:
    da = target_mz * ppm * 1e-6
    tol = max(da, abs_da)
    return target_mz - tol, target_mz + tol


def _match_targets(sorted_mz: np.ndarray, targets: np.ndarray,
                   ppm: float, abs_da: float, available_mask: np.ndarray) -> dict[int, int | None]:
    results: dict[int, int | None] = {}
    for ti, t in enumerate(targets):
        lo, hi = _ppm_window(t, ppm, abs_da)
        j = bisect_left(sorted_mz, t)
        best_idx = None
        best_delta = float("inf")
        for k in (j, j-1, j+1, j-2, j+2, j-3, j+3):
            if 0 <= k < len(sorted_mz):
                mz_k = sorted_mz[k]
                if available_mask[k] and (lo <= mz_k <= hi):
                    delta = abs(mz_k - t)
                    if delta < best_delta:
                        best_delta = delta
                        best_idx = k
        results[ti] = best_idx
    return results


def _generate_charge_series(neutral_mass: float, z_min: int, z_max: int) -> pd.DataFrame:
    z = np.arange(z_min, z_max + 1, dtype=int)
    mz = (neutral_mass + z * PROTON_MASS) / z
    return pd.DataFrame({"z": z, "target_mz": mz})


def assign_ms1_peaks(raw_df: pd.DataFrame, deconv_peaks_df: pd.DataFrame,
                     meta: dict | None = None) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Assign raw MS1 peaks to detected neutral masses (from deconvoluted spectrum) by charge-series matching.

    Returns:
      assigned_raw: raw_df with columns [assigned_mass, assigned_charge, is_assigned]
      assignments_summary: one row per accepted neutral mass with metadata, includes SNR
    """
    raw_df = raw_df.sort_values("mz").reset_index(drop=True)
    mz_arr = raw_df["mz"].to_numpy()
    inten_arr = raw_df["intensity"].to_numpy()
    available = np.ones(len(raw_df), dtype=bool)

    assigned_mass = np.full(len(raw_df), np.nan)
    assigned_z    = np.full(len(raw_df), np.nan)

    summary_rows = []

    for r in deconv_peaks_df.itertuples(index=False):
        mass = float(r.mass)
        mass_intensity = float(r.intensity)
        mass_snr = float(getattr(r, "snr", np.nan))  # <-- carry SNR into the summary

        series = _generate_charge_series(mass, Z_MIN, Z_MAX)
        targets = series["target_mz"].to_numpy()

        matches = _match_targets(mz_arr, targets, PPM_TOL, ABS_DA_TOL, available_mask=available)

        matched_indices = []
        matched_z_list  = []
        matched_mz_list = []

        for ti, k in matches.items():
            if k is not None:
                matched_indices.append(k)
                matched_z_list.append(int(series.iloc[ti]["z"]))
                matched_mz_list.append(mz_arr[k])

        if len(matched_indices) >= MIN_MATCHED_CHARGE_STATES:
            # accept and mark assigned
            for idx, z_val in zip(matched_indices, matched_z_list):
                if available[idx]:
                    available[idx] = False
                    assigned_mass[idx] = mass
                    assigned_z[idx] = z_val

            frac_intensity_removed = (
                float(np.sum(inten_arr[matched_indices])) / float(np.sum(inten_arr))
                if inten_arr.sum() > 0 else 0.0
            )

            row = {
                "neutral_mass": mass,
                "deconv_intensity": mass_intensity,
                "snr": mass_snr,  # <-- new column in assignments_summary
                "n_matches": len(matched_indices),
                "matched_z_list": json.dumps(matched_z_list),
                "matched_mz_list": json.dumps([round(float(x), 1) for x in matched_mz_list]),
                "ppm_tol": PPM_TOL,
                "abs_da_tol": ABS_DA_TOL,
                "fraction_total_intensity_captured": frac_intensity_removed
            }
            # attach filename metadata if available
            if meta:
                row.update({
                    "bin": meta.get("bin"),
                    "experiments_ids": meta.get("experiments_ids"),
                    "controls_ids": meta.get("controls_ids"),
                    "regulation": meta.get("regulation"),
                    "replicate": meta.get("replicate"),
                    "source_file": meta.get("source_file"),
                })
            summary_rows.append(row)

    assigned_raw = raw_df.copy()
    assigned_raw["assigned_mass"] = assigned_mass
    assigned_raw["assigned_charge"] = assigned_z
    assigned_raw["is_assigned"] = ~np.isnan(assigned_mass)

    assignments_summary = pd.DataFrame(summary_rows).sort_values(
        "deconv_intensity", ascending=False).reset_index(drop=True)

    # if meta present, also add bin to assigned_raw (useful downstream)
    if meta and "bin" in meta:
        assigned_raw["bin"] = meta["bin"]

    return assigned_raw, assignments_summary


# ============================================================
# ----------------------  PLOTTING  --------------------------
# ============================================================

def plot_neutral_mass_spectrum(deconv_peaks_df: pd.DataFrame, out_dir: str,
                               filename: str = "neutral_mass_spectrum.png"):
    if deconv_peaks_df.empty:
        print("No neutral masses to plot.")
        return
    masses = deconv_peaks_df["mass"].to_numpy()
    intens = deconv_peaks_df["intensity"].to_numpy()

    plt.figure(figsize=(9, 4.5))
    plt.vlines(masses, 0, intens, linewidth=1)
    plt.xlabel("Neutral mass (Da)")
    plt.ylabel("Intensity (arb.)")
    plt.title("Neutral Mass Spectrum (detected peaks)")
    plt.tight_layout()
    out_path = os.path.join(out_dir, filename)
    plt.savefig(out_path, dpi=200)
    plt.close()
    print(f"Saved plot: {out_path}")


def plot_mirror_assigned_vs_total(assigned_raw: pd.DataFrame, out_dir: str,
                                  filename: str = "mirror_assigned_vs_total.png"):
    if assigned_raw.empty:
        print("No assigned/raw data to plot.")
        return

    mz = assigned_raw["mz"].to_numpy()
    total_int = assigned_raw["intensity"].to_numpy()
    assigned_mask = assigned_raw["is_assigned"].to_numpy(dtype=bool)
    assigned_int = np.where(assigned_mask, total_int, 0.0)

    plt.figure(figsize=(10, 5.2))
    plt.vlines(mz, 0, total_int, linewidth=0.6)                 # Top: total
    plt.vlines(mz[assigned_mask], 0, -assigned_int[assigned_mask], linewidth=0.8)  # Bottom: assigned (neg)

    ymax = total_int.max() if len(total_int) else 1.0
    ymin = -assigned_int.max() if assigned_int.any() else -0.1 * ymax
    plt.ylim(ymin * 1.05, ymax * 1.05)

    plt.xlabel("m/z")
    plt.ylabel("Intensity (arb.)")
    plt.title("Mirror Plot: Total (top) vs Assigned (bottom)")
    plt.tight_layout()
    out_path = os.path.join(out_dir, filename)
    plt.savefig(out_path, dpi=200)
    plt.close()
    print(f"Saved plot: {out_path}")


def plot_mirror_unassigned_vs_total(assigned_raw: pd.DataFrame, out_dir: str,
                                    filename: str = "mirror_unassigned_vs_total.png"):
    if assigned_raw.empty:
        print("No assigned/raw data to plot.")
        return

    mz = assigned_raw["mz"].to_numpy()
    total_int = assigned_raw["intensity"].to_numpy()
    unassigned_mask = ~assigned_raw["is_assigned"].to_numpy(dtype=bool)
    unassigned_int = np.where(unassigned_mask, total_int, 0.0)

    plt.figure(figsize=(10, 5.2))
    plt.vlines(mz, 0, total_int, linewidth=0.6)                     # Top: total
    plt.vlines(mz[unassigned_mask], 0, -unassigned_int[unassigned_mask], linewidth=0.8)  # Bottom: unassigned

    ymax = total_int.max() if len(total_int) else 1.0
    ymin = -unassigned_int.max() if unassigned_int.any() else -0.1 * ymax
    plt.ylim(ymin * 1.05, ymax * 1.05)

    plt.xlabel("m/z")
    plt.ylabel("Intensity (arb.)")
    plt.title("Mirror Plot: Total (top) vs Non-assigned (bottom)")
    plt.tight_layout()
    out_path = os.path.join(out_dir, filename)
    plt.savefig(out_path, dpi=200)
    plt.close()
    print(f"Saved plot: {out_path}")


def plot_mirror_assigned_by_protein_vs_total(
    assigned_raw: pd.DataFrame,
    out_dir: str,
    filename: str = "mirror_assigned_by_protein_vs_total.png",
    max_legend_items: int = 20
):
    if assigned_raw.empty:
        print("No assigned/raw data to plot.")
        return

    mz = assigned_raw["mz"].to_numpy()
    total_int = assigned_raw["intensity"].to_numpy()

    plt.figure(figsize=(11, 5.6))
    plt.vlines(mz, 0, total_int, linewidth=0.5)  # top: total

    df_assigned_only = assigned_raw[assigned_raw["is_assigned"]].copy()
    if df_assigned_only.empty:
        plt.xlabel("m/z")
        plt.ylabel("Intensity (arb.)")
        plt.title("Mirror Plot: Total (top) vs Assigned by Protein (bottom)")
        plt.tight_layout()
        out_path = os.path.join(out_dir, filename)
        plt.savefig(out_path, dpi=200)
        plt.close()
        print(f"Saved plot: {out_path}")
        return

    counts = (
        df_assigned_only.groupby("assigned_mass", dropna=True)["is_assigned"]
        .count()
        .sort_values(ascending=False)
    )
    proteins_in_order = counts.index.tolist()
    color_cycle = plt.rcParams['axes.prop_cycle'].by_key().get(
        'color', ['C0','C1','C2','C3','C4','C5','C6','C7','C8','C9']
    )

    legend_entries = 0
    for i, mass in enumerate(proteins_in_order):
        mask = (assigned_raw["assigned_mass"] == mass)
        mz_i = assigned_raw.loc[mask, "mz"].to_numpy()
        inten_i = assigned_raw.loc[mask, "intensity"].to_numpy()

        label = None
        if legend_entries < max_legend_items:
            label = f"{mass/1000:.2f} kDa (n={len(mz_i)})"
            legend_entries += 1

        plt.vlines(
            mz_i, 0, -inten_i,
            linewidth=0.8,
            color=color_cycle[i % len(color_cycle)],
            label=label
        )

    ymax = total_int.max() if len(total_int) else 1.0
    ymin = -df_assigned_only["intensity"].max() if len(df_assigned_only) else -0.1 * ymax
    plt.ylim(ymin * 1.05, ymax * 1.05)

    if legend_entries:
        plt.legend(title="Assigned proteins", loc="upper right", fontsize=8, ncol=1)

    plt.xlabel("m/z")
    plt.ylabel("Intensity (arb.)")
    plt.title("Mirror Plot: Total (top) vs Assigned by Protein (bottom)")
    plt.tight_layout()
    out_path = os.path.join(out_dir, filename)
    plt.savefig(out_path, dpi=200)
    plt.close()
    print(f"Saved plot: {out_path}")


# ============================================================
# ----------------------  MAIN SCRIPT  -----------------------
# ============================================================

# ---- User-configurable paths ----
RAW_MS1_CSV = r"F:/test/5__pos_1__neg_0_pos_runA.csv"        # raw MS1 (m/z, intensity)
DECONV_TXT  = r"F:/test/5__pos_1__neg_0_pos_runA_mass.txt"   # deconvoluted neutral masses (mass intensity)
OUT_DIR     = r"F:/new"

# ---- Neutral-mass peak detection parameters ----
DECONV_DETECT_PARAMS = PeakFindingParams(
    min_distance_pts=20,   # decon masses can be coarse; 2 is a good start
    min_snr=10,          # enforce minimum SNR
    smooth_window=0,      # set to 5/7 if your decon spectrum is very noisy
    # min_prominence=None, min_height=None  # auto from MAD if None
)


def main():
    os.makedirs(OUT_DIR, exist_ok=True)

    # --- 1) Load deconvoluted spectrum & detect neutral-mass peaks ---
    meta = parse_metadata_from_filename(DECONV_TXT)
    deconv_raw = load_space_separated(DECONV_TXT)
    deconv_peaks = detect_signals(deconv_raw, params=DECONV_DETECT_PARAMS)

    # attach filename metadata to neutral-mass peaks table for traceability
    deconv_peaks = deconv_peaks.assign(
        bin=meta.get("bin"),
        experiments_ids=meta.get("experiments_ids"),
        controls_ids=meta.get("controls_ids"),
        regulation=meta.get("regulation"),
        replicate=meta.get("replicate"),
        source_file=meta.get("source_file"),
    )

    # Save + plot neutral-mass detections alongside the deconv file
    in_path = Path(DECONV_TXT)
    out_detect_csv = str(in_path.with_name(in_path.stem + "_detected_signals.csv"))
    out_detect_png = str(in_path.with_name(in_path.stem + "_detected_signals.png"))
    deconv_peaks.to_csv(out_detect_csv, index=False)
    plot_spectrum_with_peaks(deconv_raw, deconv_peaks, out_png=out_detect_png)
    print(f"[Neutral-mass detection] {len(deconv_peaks)} peaks → {out_detect_csv}")
    print(f"[Neutral-mass detection] Plot saved → {out_detect_png}")
    print("Parsed filename metadata:", meta)

    # --- 2) Read raw MS1 and assign charge-series to detected masses ---
    raw_df = _read_raw_ms1(RAW_MS1_CSV)

    assigned_raw, summary = assign_ms1_peaks(raw_df, deconv_peaks, meta=meta)

    out_assigned = os.path.join(OUT_DIR, "assigned_ms1_with_peaks.csv")
    out_summary  = os.path.join(OUT_DIR, "assignments_summary.csv")
    assigned_raw.to_csv(out_assigned, index=False)
    summary.to_csv(out_summary, index=False)

    # --- 3) Plots on assignments ---
    plot_neutral_mass_spectrum(deconv_peaks, OUT_DIR, filename="neutral_mass_spectrum.png")
    plot_mirror_assigned_vs_total(assigned_raw, OUT_DIR, filename="mirror_assigned_vs_total.png")
    plot_mirror_unassigned_vs_total(assigned_raw, OUT_DIR, filename="mirror_unassigned_vs_total.png")
    plot_mirror_assigned_by_protein_vs_total(
        assigned_raw, OUT_DIR, filename="mirror_assigned_by_protein_vs_total.png", max_legend_items=20
    )

    # --- 4) Console report ---
    print("\n=== Summary ===")
    print(f"Raw MS1 peaks (rows): {len(raw_df):,}")
    print(f"Detected neutral-mass peaks: {len(deconv_peaks):,}")
    print(f"Assigned raw peaks: {int(assigned_raw['is_assigned'].sum()):,}")
    print(f"Non-assigned raw peaks: {int((~assigned_raw['is_assigned']).sum()):,}")
    print(f"Saved: {out_assigned}")
    print(f"Saved: {out_summary}")


if __name__ == "__main__":
    main()


[Neutral-mass detection] 62 peaks → F:\test\5__pos_1__neg_0_pos_runA_mass_detected_signals.csv
[Neutral-mass detection] Plot saved → F:\test\5__pos_1__neg_0_pos_runA_mass_detected_signals.png
Parsed filename metadata: {'bin': 5, 'experiments': 1, 'controls': 1, 'experiments_ids': '1', 'controls_ids': '0', 'experiments_n': 1, 'controls_n': 1, 'regulation': 'upregulated', 'replicate': 'A', 'source_file': '5__pos_1__neg_0_pos_runA_mass.txt'}
Saved plot: F:/new\neutral_mass_spectrum.png
Saved plot: F:/new\mirror_assigned_vs_total.png
Saved plot: F:/new\mirror_unassigned_vs_total.png
Saved plot: F:/new\mirror_assigned_by_protein_vs_total.png

=== Summary ===
Raw MS1 peaks (rows): 7,346
Detected neutral-mass peaks: 62
Assigned raw peaks: 1,352
Non-assigned raw peaks: 5,994
Saved: F:/new\assigned_ms1_with_peaks.csv
Saved: F:/new\assignments_summary.csv
